# Install and Imports

In [17]:
!pip install pypdf
!pip install google-generativeai
!pip install chromadb
!pip install typing

In [18]:
import requests
from pypdf import PdfReader
import os
import re
import google.generativeai as genai
from chromadb import Documents, EmbeddingFunction, Embeddings
import chromadb
from chromadb.config import Settings
from typing import List

In [19]:
import dotenv

dotenv.load_dotenv("env")

True

# Download and load PDF

In [20]:
def download_pdf(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as f:
        f.write(response.content)

def load_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text
    return text

# ToDo:
- Text splitting
- ChromaDB
- Prompt Construction

In [21]:
# TODO: Students implement text splitting function
def split_text(text, max_chunk_size=1000, overlap=200):
    """
    Split the input text into meaningful chunks.
    Returns a list of text chunks.
    """
    words = text.split()
    chunks = []
    start = 0

    while start < len(words):
        end = start + max_chunk_size
        chunk = " ".join(words[start:end])
        chunks.append(chunk)
        start = end - overlap

    return chunks

# Custom embedding function using Gemini API
class GeminiEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        gemini_api_key = os.getenv("GEMINI_API_KEY")
        genai.configure(api_key=gemini_api_key)
        model = "models/embedding-001"
        title = "Custom query"
        return genai.embed_content(model=model, content=input, task_type="retrieval_document", title=title)["embedding"]

# TODO: Students implement ChromaDB creation and querying
def create_chroma_db(documents: List[str], path: str, name: str):
    """
    Create a ChromaDB collection with the provided documents.
    Returns the database instance and name.

    Hint: Use the following to create the client:
    client = chromadb.Client(Settings(
        chroma_db_impl="duckdb+parquet",
        persist_directory=path
    ))
    """
    # Initialize Chroma client
    client = chromadb.Client()

    # Create collection
    collection = client.get_or_create_collection(name=name)

    # Add documents to collection
    collection.upsert(
      documents=documents,
      ids=[str(i) for i in range(len(documents))]
    )

    return collection, client

def get_relevant_passage(query: str, db, n_results: int):
    """
    Retrieve the most relevant passages for the given query.
    Returns a list of relevant text passages.
    """
    results = db.query(
        query_texts=[query],
        n_results=n_results
    )
    return results["documents"][0] if results["documents"] else []


# TODO: Students implement prompt construction
def make_rag_prompt(query: str, relevant_passage: str):
    """
    Construct a prompt for the generation model using the query and retrieved passage.
    Returns the formatted prompt string.
    """
    prompt = (
        f"You are a knowledgeable assistant. Use the following relevant passage to answer the user's query.\n\n"
        f"Relevant Passage:\n{relevant_passage}\n\n"
        f"Query: {query}\n\n"
        f"Provide a concise and accurate response:"
    )
    return prompt

# LLM Response Generation

In [22]:
def generate_answer(prompt: str):
    """Generate answer using Gemini Pro API"""
    gemini_api_key = os.getenv("GEMINI_API_KEY")
    if not gemini_api_key:
        raise ValueError("Gemini API Key not provided. Please provide GEMINI_API_KEY as an environment variable")
    genai.configure(api_key=gemini_api_key)
    model = genai.GenerativeModel('gemini-pro')
    result = model.generate_content(prompt)
    return result.text

# Main execution
## ToDo:
 - Chat history
 - Multiple file injest

In [23]:
def main():
    # Set up configurations
    pdf_url = "https://services.google.com/fh/files/misc/ai_adoption_framework_whitepaper.pdf"
    pdf_path = "ai_adoption_framework_whitepaper.pdf"
    db_folder = "chroma_db"
    db_name = "rag_experiment"

    # Create database directory
    if not os.path.exists(db_folder):
        os.makedirs(db_folder)

    # Download and process PDF
    download_pdf(pdf_url, pdf_path)
    pdf_text = load_pdf(pdf_path)

    # Split text into chunks
    chunked_text = split_text(pdf_text)

    # Create and set up database
    db_path = os.path.join(os.getcwd(), db_folder)
    db, _ = create_chroma_db(chunked_text, db_path, db_name)

    # Process user query
    query = input("Please enter your query: ")
    relevant_text = get_relevant_passage(query, db, n_results=3)

    # Generate and display answer
    if relevant_text:
        final_prompt = make_rag_prompt(query, "".join(relevant_text))
        answer = generate_answer(final_prompt)
        print("\nGenerated Answer:", answer)
    else:
        print("No relevant information found for the given query.")

if __name__ == "__main__":
    main()

Please enter your query: give me the main idea

Generated Answer: The AI Adoption Framework emphasizes six themes (Learn, Lead, Access, Scale, Automate, Secure) that determine an organization's readiness for AI adoption and categorize them into three maturity phases (Tactical, Strategic, Transformational). Each phase offers opportunities for growth and advancement as organizations strive to leverage AI for business value.
